In [3]:
# import packages
import scipy.io as sio
import matlab.engine
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# %matplotlib inline

ImportError: No module named 'matlab'

In [2]:
# run matlab script to put data together
eng = matlab.engine.connect_matlab()
eng.cd(r'C:\Users\ishparii\dev\SkyFall_GLM', nargout=0)
eng.ls(nargout=0)

data_CF = eng.TrainingDataSetup([],[],10,0) #location -use all; subjID - use all; n jitter - 10; condition - healthy

NameError: name 'matlab' is not defined

In [ ]:
data_CF_np = np.array(data_CF)
data_CF_df = pd.DataFrame(data_CF_np)

print(data_CF_df.shape)
data_CF_df.head()


In [ ]:
# location: 0 - N/A; 1 - pouch; 2 - pocket; 3 - hand
# subj_code: 0 - amputee; 1 - healthy
# label: 1 - slip; 2 - trip; 3 - right; 4 - left
data_CF_df=data_CF_df.rename(columns = {0:'subj_id', 1:'location', 2:'subj_code', 3:'label'})

# save data to file
data_CF_df.to_csv('data_CF.csv')

data_CF_df.head()

In [ ]:
data_AF = eng.TrainingDataSetup([],[],10,1) #location -use all; subjID - use all; n jitter - 10; condition - amputees
# data_AF

In [ ]:
data_AF_np = np.array(data_AF)
data_AF_df = pd.DataFrame(data_AF_np)

print(data_AF_df.shape)
data_AF_df.head()

In [ ]:
# location: 0 - N/A; 1 - pouch; 2 - pocket; 3 - hand
# subj_code: 0 - amputee; 1 - healthy
# label: 1 - slip; 2 - trip; 3 - right; 4 - left
data_AF_df=data_AF_df.rename(columns = {0:'subj_id', 1:'location', 2:'subj_code', 3:'label'})

# save data to file
data_AF_df.to_csv('data_AF.csv')

data_AF_df.head()

In [ ]:
# read data from csv files
data_CF_df = pd.read_csv('data_CF.csv', index_col=0)
data_CF_df.head()

data_AF_df = pd.read_csv('data_AF.csv', index_col=0)
data_AF_df.head()

In [ ]:
# changing labels: 1 - fall; 9 - non-fall 
data_CF_df.loc[(data_CF_df.label == 4) | (data_CF_df.label == 3) | (data_CF_df.label == 2), ['label']] = 1
data_AF_df.loc[(data_AF_df.label == 4) | (data_AF_df.label == 3) | (data_AF_df.label == 2), ['label']] = 1

# change amputee subjects ids
max_id_CF = data_CF_df.subj_id.max()
print(max_id_CF)
data_AF_df.subj_id = data_AF_df.subj_id + max_id_CF
print("Amputee subjects id: ", data_AF_df.subj_id.unique())
print()
print("Unique labels: ", data_AF_df.label.unique())

In [ ]:
# training and testing data split

# location: 0 - N/A; 1 - pouch; 2 - pocket; 3 - hand
# subj_code: 0 - amputee; 1 - healthy
# label: 1 - slip; 2 - trip; 3 - right; 4 - left
data_train = data_CF_df.loc[data_CF_df['location'] == 2] # pocket data
data_test = data_AF_df.loc[data_AF_df['location'] == 2]

#data_train = data_CF_df[(data_CF_df.location == 1) & ((data_CF_df.label>0) & (data_CF_df.label<=4))]
#data_test = data_AF_df[(data_AF_df.location == 1) & ((data_AF_df.label>0) & (data_AF_df.label<=4))]

print(data_train.shape)
print(data_test.shape)

In [ ]:
# mix data
import random
from sklearn.utils import shuffle

data = data_CF_df
full_data = data.append(data_AF_df)

# location: 0 - N/A; 1 - pouch; 2 - pocket; 3 - hand
# subj_code: 0 - amputee; 1 - healthy
# label: 1 - slip; 2 - trip; 3 - right; 4 - left
# full_data = full_data[((full_data.location == 1) 
# #                        | (full_data.location == 3)
#                       )] # data for pouch + hand
print("Data size: ", full_data.shape)
print()

subj_ids = np.sort(np.array(full_data.subj_id.unique()))
print("All subjects ids: ",subj_ids)

# first and last 2 subjects ids for testing
test_subjects = subj_ids[np.r_[0:2, -2:0]]
print("Subjects for testing: ",test_subjects)

train_subjects = np.setdiff1d(subj_ids,test_subjects)
print("Subjects for training: ",train_subjects)

data_train = full_data.loc[full_data['subj_id'].isin(train_subjects)]
data_test = full_data.loc[full_data['subj_id'].isin(test_subjects)]
# group data by subject
#groups = full_data.groupby('subj_id')
#groups.head()

#random.shuffle(groups)

#for g, grp in groups:
#    print (grp)

#data = pd.DataFrame(groups)
#print(data.shape)
#data.head()

#data_train, data_test = train_test_split(data, test_size=0.3)
print("Training data size: ", data_train.shape)
print("Testing data size: ",data_test.shape)

In [ ]:
print("Unique labels in training set: ", data_train.label.unique())
print()
print("Unique labels in testing set: ", data_test.label.unique())

In [ ]:
# list of features
features = list(range(4,1215))
features = np.array(features)

In [ ]:
# handpicking features

# feature groups:
#       1 - Raw Signal Statistics
#       2 - Raw Signal Correlation Coefficients
#       3 - Raw Signal 5s FFT bins
#       4 - Raw Signal 1s FFT bins
#		5 - Derivative Statistics
#		6 - Derivative 5s FFT bins
#		7 - Derivative 1s FFT bins
#		8 - Resultant Vector and Magnitude 
#		9 - Angle Statistics (ArcTan)
#		10 - Entropies
#		11 - Raw Signal Cross Products
#		12 - Derivative Cross Products
#		13 - Raw Signal Statistics on 1s FFT bins
#		14 - Raw Signal Entropies on 1s FFT bins
#		15 - Raw Signal Statistics on 1s binned signal energy
#		16 - Derivative Statistics on 1s FFT bins
#		17 - Derivative Entropies on 1s FFT bins
#		18 - Barometer

features_to_use = eng.getFeatureInds(matlab.logical([1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]))

features_to_use = np.array(features_to_use)
features_to_use = [y for x in features_to_use for y in x] # flatten array
features_to_use = np.array(features_to_use)

features = features[features_to_use]
print(features)
print(len(features))

In [ ]:
# features analysis
from sklearn.ensemble import ExtraTreesClassifier

forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)

X = data_train[features]
Y = data_train['label']

forest.fit(X, Y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

In [ ]:
# normalize feature importances

importances = np.array(importances)
importances /= importances.max()

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

In [ ]:
# select features > 10 % importance
features = [i for i,j in zip(features,importances) if j > 0.1]

print ("Selected number of features: %d" % len(features))

for f in range(len(features)):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import LeaveOneGroupOut
from sklearn import metrics, svm, neighbors, linear_model, tree
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# setting data for training
X = data_train[features] # data for training
Y = data_train['label'] # target labels
groups = data_train['subj_id']

In [ ]:
# Different models to try
#       Model name ---------------------------------------------------------------------
#      Parameters ------------------------------------------                           |
#     Classifier -----------                               |                           |
#                          |                               |                           |
#                          v                               v                           |
models = [#[tree.DecisionTreeClassifier(), {'min_samples_split': [2, 4, 6, 8, 10],#    |
#                                            'min_samples_leaf': [1, 5, 10, 15, 20],#     v
#                                            'max_depth': [10, 20, 30, 40, 50]},       "Decision Tree"]
          ]

# models.append([linear_model.LogisticRegression(), {'C': [1e+4, 1e+3, 1e+2, 1e+1, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]}, "Logistic Regression with Ridge Penalty"])

# models.append([linear_model.LogisticRegression(penalty='l1'), {'C': [100, 10, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]}, "Logistic Regression with Lasso Penalty"])

# LinearSVC
#models.append([svm.SVC(kernel='rbf', class_weight={1:10,9:1}), {#'kernel': ['rbf'], # class_weight="balanced"; tol
                          # 'gamma': [1e+4, 1e+3, 1e+2, 1e+1, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6], 
                           #'C': [1e+4, 1e+3, 1e+2, 1e+1, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]}, "SVM rbf"])

# models.append([neighbors.KNeighborsClassifier(), {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15]}, "K-Nearest Neighbors"])



models.append([RandomForestClassifier(), {'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200],
                                           'max_depth': [1, 10, 20, 30, 40, 50],
                                           'min_samples_leaf': [1, 5, 10, 15, 20],
                                          'min_samples_split': [2, 4, 6, 8, 10]
                                           }, "Random Forest"])

models_with_best_params = []

# cross-validation strategy
# cv = 10 # for 10-folds cross-validation

# Leave One Group Out
logo = LeaveOneGroupOut()
cv = logo.get_n_splits(X,Y,groups)


for model in models:
    clf = GridSearchCV(model[0], model[1], cv=cv)
    clf.fit(X, Y)
    best_params = clf.best_params_
    best_estimator = clf.best_estimator_
    
    model_with_best_params = [best_estimator, best_params, model[2]]
    
    models_with_best_params.append(model_with_best_params)
    
    print(model[2], ": ")
    print("Best score for ", model[2], ":", clf.best_score_)
    print()
    print("Best parameters for ", model[2], " found on development set:", best_params)
    print()
    print("Best estimator for ", model[2], " model:", best_estimator)
    print()
    print()

    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))
    print()

print(models_with_best_params)


In [ ]:
#fitting models to test_data
for model in models_with_best_params:
    classifier = model[0]
    classifier.fit(X, Y)
    score = classifier.score(data_test[features], data_test['label'])
    print("Prediction accuracy for", model[2], "model is", score)
    expected = data_test['label']
    predicted = classifier.predict(data_test[features])

    print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted, digits=5)))
    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [ ]:
# count falls and non-falls in test set
labels_np = np.array(expected)
unique, counts = np.unique(labels_np, return_counts=True)

print (np.asarray((unique, counts)).T)